# Invasive Species Monitoring

First we need to create a directory structure and get a small sample of training and validation data.

In [70]:
import os
import numpy as np
import pandas as pd
cwd = os.getcwd()

main_path = "data\\"
data_path ="data\\sample\\"
#data_path = "data/dogscats/"
test_path = data_path + "test\\"

In [131]:
main_path = "data/"
data_path ="data/sample/"
#data_path = "data/"
train_path = data_path + "train/"
valid_path = data_path + "valid/"
test_path = data_path + "test/"

In [43]:
#import training labels

training_labels = pd.read_csv(main_path+'train_labels.csv',index_col=0)
training_labels.head(3)
type(training_labels.index[0])

numpy.int64

In [64]:
import glob
import re
import shutil

#get image names
image_names = [filename for filename in glob.glob(main_path + 'train/*')]

#if windows replace backslashes (on linux this needs to be commented out)
image_names = list(map(lambda x: x.replace("\\","/"), image_names))

#join names with labels
pattern = re.compile("\d+")
image_ids = [pattern.search(x).group() for x in image_names if pattern.search(x) != None]
count_non_images = len([pattern.search(x) for x in image_names if pattern.search(x) == None])
image_df = pd.DataFrame({'id':image_ids,'filename':image_names[:-2]})
image_df.set_index('id',inplace=True)
image_df.index = image_df.index.astype(int)
image_df = image_df.join(training_labels,how='left')

invasive_filenames = image_df[image_df['invasive']==1]['filename']
not_invasive_filenames = image_df[image_df['invasive']==0]['filename']

invasive_filenames[:3]
for filename in invasive_filenames:
   shutil.move(filename,main_path+'train/Invasive/')
    
for filename in not_invasive_filenames:
   shutil.move(filename,main_path+'train/Not-invasive/')

2


In [86]:
#Create amended paths
invasive_filenames_m = [x.replace('train/','train/Invasive/') for x in invasive_filenames]
not_invasive_filenames_m = [x.replace('train/','train/Not-invasive/') for x in not_invasive_filenames]
#not_invasive_filenames_m[:3]

['data/train/Not-invasive/1.jpg',
 'data/train/Not-invasive/10.jpg',
 'data/train/Not-invasive/1001.jpg']

Create directory structure for validation

In [69]:
import os
import errno

def make_sure_path_exists(path):
    try:
        os.makedirs(path)
    except OSError as exception:
        if exception.errno != errno.EEXIST:
            raise
            
make_sure_path_exists(main_path+'valid/')
make_sure_path_exists(main_path+'valid/Invasive')
make_sure_path_exists(main_path+'valid/Not-invasive')

Move 10% data for validation

In [91]:
pct=0.1
move_invasive_filenames = np.random.choice(invasive_filenames_m,int(pct*len(invasive_filenames_m)),replace=False)
print("Invasive images moved to validation: " + str(len(move_invasive_filenames)))
move_not_invasive_filenames = np.random.choice(not_invasive_filenames_m,int(pct*len(not_invasive_filenames_m)),replace=False)
print("Not-invasive images moved to validation: " + str(len(move_not_invasive_filenames)))

for filename in move_invasive_filenames:
   shutil.move(filename,main_path+'valid/Invasive/')
    
for filename in move_not_invasive_filenames:
   shutil.move(filename,main_path+'valid/Not-invasive/')

Invasive images moved to validation: 144
Not-invasive images moved to validation: 84


In [95]:
#Amend validation filenames
#Create amended paths
valid_invasive_filenames_m = [x.replace('train/Invasive/','valid/Invasive/') for x in move_invasive_filenames]
valid_not_invasive_filenames_m = [x.replace('train/Not-invasive/','valid/Not-invasive/') for x in move_not_invasive_filenames]
#valid_invasive_filenames_m[:3]
#valid_not_invasive_filenames_m[:3]

['data/valid/Not-invasive/45.jpg',
 'data/valid/Not-invasive/1637.jpg',
 'data/valid/Not-invasive/291.jpg']

Copy 20% data to sample 

In [96]:
make_sure_path_exists(main_path+'sample/valid/')
make_sure_path_exists(main_path+'sample/valid/Invasive')
make_sure_path_exists(main_path+'sample/valid/Not-invasive')
make_sure_path_exists(main_path+'sample/train/')
make_sure_path_exists(main_path+'sample/train/Invasive')
make_sure_path_exists(main_path+'sample/train/Not-invasive')
make_sure_path_exists(main_path+'sample/test/Unknown')

In [98]:
def copy_files_to_sample(prefix,suffix,pct=0.1,windows=False):
    image_names = [filename for filename in glob.glob(main_path + prefix + suffix + '*')]
    #if windows replace backslashes (on linux this needs to be commented out)
    if (windows):
        image_names = list(map(lambda x: x.replace("\\","/"), image_names))

    copy_image_names = np.random.choice(image_names,int(pct*len(image_names)),replace=False)
    print(prefix + ", " + suffix + " images copied: " + str(len(copy_image_names)))
        
    for filename in copy_image_names:
       shutil.copy(filename,main_path+'sample/' + prefix + suffix)

In [97]:
pct=0.2
windows=True

In [99]:
#get training Invasive image names
copy_files_to_sample('train/','Invasive/',pct=pct,windows=windows)
copy_files_to_sample('train/','Not-invasive/',pct=pct,windows=windows)
copy_files_to_sample('valid/','Invasive/',pct=pct,windows=windows)
copy_files_to_sample('valid/','Not-invasive/',pct=pct,windows=windows)
copy_files_to_sample('test/','Unknown/',pct=pct,windows=windows)


train/, Invasive/ images copied: 260
train/, Not-invasive/ images copied: 152
valid/, Invasive/ images copied: 28
valid/, Not-invasive/ images copied: 16
test/, Unknown/ images copied: 306


Load utils and initialize custom Tensorflow session for Keras

In [102]:
%matplotlib inline
from imp import reload
import utils; reload(utils)
from utils import *

In [4]:
!nvidia-smi

Thu Aug 17 05:32:30 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.48                 Driver Version: 367.48                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | E486:00:00.0     Off |                    0 |
| N/A   40C    P0    71W / 149W |     64MiB / 11439MiB |      1%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [111]:
batch_size=4
make_sure_path_exists(main_path+'results/')

# Finetune and train

In [104]:
import vgg16; reload(vgg16)
from vgg16 import Vgg16

In [106]:
vgg = Vgg16()

L:\Data Science\GIT\Invasive-Species-Monitoring\vgg16.py:105: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
L:\Data Science\GIT\Invasive-Species-Monitoring\vgg16.py:105: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
L:\Data Science\GIT\Invasive-Species-Monitoring\vgg16.py:105: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(256, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))
L:\Data Science\GIT\Invasive-Species-Monitoring\vgg16.py:105: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu")`
  model.add(Convolution2D(filters, 3, 3, activation='relu'))


In [107]:
batches = vgg.get_batches(data_path+'train',batch_size=batch_size)
val_batches = vgg.get_batches(data_path+'valid', batch_size=batch_size)
vgg.finetune(batches)

Found 412 images belonging to 2 classes.
Found 44 images belonging to 2 classes.


vgg.model.load_weights(main_path+'results/ft01.h5')

In [108]:
batches_per_epoch_div = 1
epochs_to_run=1

In [109]:
vgg.model.optimizer.lr=0.001
vgg.fit(batches,val_batches,nb_epoch=epochs_to_run,steps_per_epoch=(batches.samples // batch_size) // batches_per_epoch_div, validation_steps=val_batches.samples // batch_size)

L:\Data Science\GIT\Invasive-Species-Monitoring\vgg16.py:218: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  validation_data=val_batches, validation_steps=validation_steps)
L:\Data Science\GIT\Invasive-Species-Monitoring\vgg16.py:218: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_steps=11, steps_per_epoch=103, epochs=1, validation_data=<keras.pre...)`
  validation_data=val_batches, validation_steps=validation_steps)


Epoch 1/1
103/103 [==============================] - 29s - loss: 0.7175 - acc: 0.7597 - val_loss: 0.6446 - val_acc: 0.7955


The computation is run on NVIDIA Tesla K80 with around 11GB RAM. As seen below Tensorflow session keeps the memory allocation.

In [11]:
!nvidia-smi

Thu Aug 17 05:41:14 2017       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 367.48                 Driver Version: 367.48                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | E486:00:00.0     Off |                    0 |
| N/A   67C    P0    76W / 149W |   9477MiB / 11439MiB |    100%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [112]:
vgg.model.save_weights(main_path+'results/ft01.h5')

In [137]:
vgg.model.optimizer.lr=0.0003
vgg.fit(batches,val_batches,nb_epoch=epochs_to_run,steps_per_epoch=(batches.samples // batch_size) // batches_per_epoch_div, validation_steps=val_batches.samples // batch_size)
vgg.model.save_weights(main_path+'results/ft02.h5')

L:\Data Science\GIT\Invasive-Species-Monitoring\vgg16.py:218: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  validation_data=val_batches, validation_steps=validation_steps)
L:\Data Science\GIT\Invasive-Species-Monitoring\vgg16.py:218: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_steps=11, steps_per_epoch=103, epochs=1, validation_data=<keras.pre...)`
  validation_data=val_batches, validation_steps=validation_steps)


Epoch 1/1
103/103 [==============================] - 27s - loss: 0.4881 - acc: 0.8301 - val_loss: 0.9108 - val_acc: 0.7045


In [139]:
vgg.model.optimizer.lr = 0.0001

In [140]:
vgg.fit(batches,val_batches,nb_epoch=epochs_to_run,steps_per_epoch=(batches.samples // batch_size) // batches_per_epoch_div, validation_steps=val_batches.samples // batch_size)
vgg.model.save_weights(main_path+'results/ft03.h5')

L:\Data Science\GIT\Invasive-Species-Monitoring\vgg16.py:218: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  validation_data=val_batches, validation_steps=validation_steps)
L:\Data Science\GIT\Invasive-Species-Monitoring\vgg16.py:218: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., validation_steps=11, steps_per_epoch=103, epochs=1, validation_data=<keras.pre...)`
  validation_data=val_batches, validation_steps=validation_steps)


Epoch 1/1
103/103 [==============================] - 27s - loss: 0.3849 - acc: 0.8859 - val_loss: 0.9568 - val_acc: 0.7955


In [ ]:
vgg.model.optimizer.lr = 0.00003
vgg.fit(batches,val_batches,nb_epoch=epochs_to_run,steps_per_epoch=(batches.samples // batch_size) // batches_per_epoch_div, validation_steps=val_batches.samples // batch_size)
vgg.model.save_weights(main_path+'results/ft04.h5')

In [ ]:
vgg.model.optimizer.lr = 0.000001
vgg.fit(batches,val_batches,nb_epoch=epochs_to_run,steps_per_epoch=(batches.samples // batch_size) // batches_per_epoch_div, validation_steps=val_batches.samples // batch_size)
vgg.model.save_weights(main_path+'results/ft05.h5')

#Windows version
vgg.model.load_weights(main_path+'results\\ft05.h5')

Score using roc_auc_score

In [141]:
from sklearn.metrics import roc_auc_score
val_batches2 = vgg.get_batches(valid_path,shuffle=False,batch_size=batch_size,class_mode=None)
val_probs = vgg.model.predict_generator(val_batches2,steps = val_batches2.samples // batch_size, verbose=1)
val_labels = val_batches2.classes
print("ROC AUC score on validation data: " + str(roc_auc_score(val_labels,val_probs[:,1])))

Found 44 images belonging to 2 classes.
11/11 [==============================] - 2s     
ROC AUC score on validation data: 0.904017857143


In [ ]:
test_batches = vgg.get_batches(test_path,shuffle=False,batch_size=batch_size,class_mode=None)
print(test_batches.samples)

In [ ]:
prediction = vgg.model.predict_generator(test_batches, steps = test_batches.samples // (batch_size), verbose=1)

In [ ]:
filenames = test_batches.filenames
print(len(filenames))

In [ ]:
print(prediction.shape)
print(prediction[:5])
prediction_trim = prediction[:len(filenames)]
print(prediction_trim.shape)

In [ ]:
prediction_clip = np.clip(prediction_trim,0.02,0.98)
prediction_clip[:5]

In [ ]:
filenames[:5]

In [ ]:
#Windows version
#from PIL import Image
#Image.open(test_path.replace("/","\\")+filenames[0])

In [ ]:
from PIL import Image
#print(prediction[0])
print(filenames[0])
Image.open(test_path+filenames[0])

In [ ]:
isdog = prediction_clip[:,1]
isdog[:5]

In [ ]:
import re
pattern = re.compile('\d+')
ids = [pattern.search(x).group() for x in filenames]
ids[-5:]


In [ ]:
print(len(ids))
print(len(isdog))

In [ ]:
def plot_idx(idx, titles=None):
    plots([image.load_img(data_path + 'test/' + i) for i in idx], titles=titles)
n_view=8

df = pd.DataFrame({'id':ids,'label':isdog})
df['id'] = df['id'].astype(int)
df = df.set_index('id')
print(df.head())
f_names = permutation(filenames)[:n_view]
idx = [pattern.search(x).group() for x in f_names]
plot_idx(f_names,list(df.loc[np.array(idx).astype(int),'label']))


In [ ]:
f_names = permutation(filenames)[:n_view]
idx = [pattern.search(x).group() for x in f_names]
plot_idx(f_names,list(df.loc[np.array(idx).astype(int),'label']))

In [ ]:
f_names = permutation(filenames)[:n_view]
idx = [pattern.search(x).group() for x in f_names]
plot_idx(f_names,list(df.loc[np.array(idx).astype(int),'label']))

In [ ]:
type(df.index[0])

In [ ]:
import pandas as pd
subm_df = pd.DataFrame({'id':ids,'label':isdog})
subm_df.head()

In [ ]:
subm_df.tail()

In [ ]:
subm_df['id'] = subm_df['id'].astype(int)
subm_df['label'] = np.round(subm_df['label'],5)
print(subm_df.info())
print(subm_df.head())

In [ ]:
subm_df.to_csv(main_path + 'results/' + 'subm011.csv',index=False, float_format='%.5f')

In [ ]:
from IPython.display import FileLink
FileLink('data/dogscats/results/' + 'subm011.csv')

In [ ]:
subm_df = pd.read_csv(main_path + 'results/' + 'subm002.csv')
print(subm_df.info())
subm_df.head()

In [ ]:
subm_df.loc[:,'label'] = 1 - subm_df['label']
#subm_df.columns = ['iid','label']
subm_df.loc[:,'label'] = np.round(subm_df['label'],5)
subm_df.sort_values('id',inplace=True)
subm_df.head()

In [ ]:
subm_df.to_csv(main_path + 'results/' + 'subm005.csv',index=False)

In [ ]:
from IPython.display import FileLink
FileLink('data/dogscats/results/' + 'subm005.csv')

# Visualize results

In [ ]:
vgg.model.load_weights(main_path+'results/ft04.h5')
batch_size=100
val_batches2 = vgg.get_batches(data_path+'valid', shuffle=False, batch_size=batch_size, class_mode=None)
val_probs = vgg.model.predict_generator(val_batches2, steps = val_batches2.samples // (batch_size),verbose=1)

In [ ]:
val_labels = val_batches2.classes
val_filenames = val_batches2.filenames

In [ ]:
val_isCat_probs = val_probs[:,0]
val_preds = np.round(1-val_isCat_probs)
val_isCat_probs[:5]

In [ ]:
val_preds[:5]

In [ ]:
#Number of images to check
n_view = 4

Helped function for plotting images and probabilities

In [ ]:
def plot_idx(idx, titles=None):
    plots([image.load_img(data_path + 'valid/' + val_filenames[i]) for i in idx], titles=titles)

Plot correct predictions

In [ ]:
correct = np.where(val_preds == val_labels)[0]
idx = permutation(correct)[:n_view]
plot_idx(idx,val_isCat_probs[idx])

Plot a few incorrect predictions

In [ ]:
incorrect = np.where(val_preds != val_labels)[0]
idx = permutation(incorrect)[:n_view]
plot_idx(idx,val_isCat_probs[idx])

Plot confident predictions for invasive

In [ ]:
correct_cats = np.where((val_preds==0) & (val_preds==val_labels))[0]
most_correct_cats = np.argsort(val_isCat_probs[correct_cats])[-n_view:]
plot_idx(correct_cats[most_correct_cats], val_isCat_probs[correct_cats][most_correct_cats])

Plot confident predictions for not-invasive

In [ ]:
correct_dogs = np.where((val_preds==1) & (val_preds==val_labels))[0]
most_correct_dogs = np.argsort(val_isCat_probs[correct_dogs])[:n_view]
plot_idx(correct_dogs[most_correct_dogs], val_isCat_probs[correct_dogs][most_correct_dogs])

Plot incorrectly predicted invasive

In [ ]:
incorrect_cats = np.where((val_preds==0) & (val_preds != val_labels))[0]
most_incorrect_cats = np.argsort(val_isCat_probs[incorrect_cats])[::-1][:n_view]
plot_idx(incorrect_cats[most_incorrect_cats], val_isCat_probs[incorrect_cats][most_incorrect_cats])

Plot incorrectly predicted not-invasive

In [ ]:
incorrect_dogs = np.where((val_preds==1) & (val_preds != val_labels))[0]
most_incorrect_dogs = np.argsort(val_isCat_probs[incorrect_dogs])[::-1][:n_view]
plot_idx(incorrect_dogs[most_incorrect_dogs], val_isCat_probs[incorrect_dogs][most_incorrect_dogs])

Plot the most uncertain predictions

In [ ]:
most_uncertain = np.argsort(np.abs(val_isCat_probs-0.5))
plot_idx(most_uncertain[:n_view], val_isCat_probs[most_uncertain][:n_view])

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
cm = confusion_matrix(val_labels,val_preds)
plot_confusion_matrix(cm, val_batches2.class_indices)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(val_labels,val_preds))